## 한국어 언어모델 학습 및 다중과제 튜닝

####KoGPT-2 기반의 챗봇 실습

In [1]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!apt-get install git-lfs

Detected operating system as Ubuntu/jammy.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.4.12
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages will be upgraded:
  git-lfs
1 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Need to get 7,420 kB of archives.
After this operation, 6,051 kB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu jammy/main amd64 git-lfs amd64 3.5.1 [7,420 kB]
Fetched 7,420 kB in 1s (10.6 MB/s)
(Reading database ... 121757 files and directories c

In [2]:
!git lfs install
!git clone https://huggingface.co/taeminlee/kogpt2

Git LFS initialized.
Cloning into 'kogpt2'...
remote: Enumerating objects: 56, done.
remote: Total 56 (delta 0), reused 0 (delta 0), pack-reused 56
Unpacking objects: 100% (56/56), 1.53 MiB | 3.19 MiB/s, done.
Filtering content: 100% (3/3), 1.41 GiB | 42.32 MiB/s, done.


In [3]:
!pip install transformers

In [4]:
import torch
from tokenizers import SentencePieceBPETokenizer
from transformers import GPT2Config, GPT2LMHeadModel

tokenizer = SentencePieceBPETokenizer("/content/kogpt2/vocab.json", "/content/kogpt2/merges.txt")

config = GPT2Config(vocab_size=50000)
model = GPT2LMHeadModel(config)

model_dir = '/content/kogpt2/pytorch_model.bin'

model.load_state_dict(torch.load(model_dir, map_location='cpu'), strict=False) # cuda 대신 cpu로

_IncompatibleKeys(missing_keys=[], unexpected_keys=['transformer.h.0.attn.bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.2.attn.bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.3.attn.bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.5.attn.bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.6.attn.bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.7.attn.bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.bias', 'transformer.h.11.attn.masked_bias'])

In [5]:
tokenized_text = tokenizer.encode('이순신은 조선 중기의 무신이다.', add_special_tokens=True)
print(tokenized_text)
print(tokenized_text.tokens)
print(tokenized_text.ids)
# 토크나이저 테스트 해보고 잘 되는 것 확인

Encoding(num_tokens=7, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['▁이순', '신은', '▁조선', '▁중기의', '▁무신', '이다', '.']
[10925, 6647, 1117, 40249, 39793, 128, 47440]


In [6]:
import torch
torch.manual_seed(42)

input_ids = torch.tensor(tokenizer.encode("이순신은", add_special_tokens=True).ids).unsqueeze(0)

output_sequences = model.generate(input_ids=input_ids, do_sample=True, max_length=100, num_return_sequences=3)
for generated_sequence in output_sequences:
    generated_sequence = generated_sequence.tolist()
    print("GENERATED SEQUENCE : {0}".format(tokenizer.decode(generated_sequence, skip_special_tokens=True)))

# 단어 생성도 잘되는 것 확인

# 다음은 데이터셋을 준비

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GENERATED SEQUENCE : 이순신은 “그 동안 은녀로 계셨던 여러분과 함께 하고 싶었습니다.</s><s> 두 사람 모두 30년 이상의 긴 호흡을 맞춘 첫사랑이라고.</s><s> 이들은 서로의 마음을 확인하는 '의자놀이'를 하며 서로를 달랬다.</s><s> 두 사람은 "내가 먼저 미안해 하는 것 같아 미안하다.</s><s> 하지만, 좋은 남자로 시작해 보자는 뜻에서 (유)남편의 제안을 받아들였으니 결혼해서 잘 살자는 바람"이라고 말하며 진심 어린 소감을 전했다.</s>
GENERATED SEQUENCE : 이순신은 자신의 아들이자 동무인 최숙향(김향수 분)를 만나러 가지만 아들 최충헌의 죽음과 함께 동무들 앞에서 죽음을 맞이하게 된다.</s><s> 한재석 측 "유리한 해명 불구 해명글 게재는 아냐"</s><s> 사진퍼가기 이용안내</s><s> 배우 한재석이 연예계 대표 ‘남남갈등’의 원인을 밝혔다.</s><s> 그는 앞서 KBS 2FM 라디오 '오후의 발견(이하 '오후의 발견')'
GENERATED SEQUENCE : 이순신은 "내가 나 때문에 내 아들을 잃은 건 아닐까?"</s><s> 이 때문에 박 씨는 '오블리비'(오블리비언을 위한 몸값)를 받으려고 한 것으로 보였다.</s><s> 제작진은 "그녀가 오블리비언을 통해 자신의 삶을 보았을 때 자신의 딸들이 어떤 삶을 산다고 믿었을까도 생각한다"고 말했다.</s><s> "오블리비언을 하면서 딸이 스스로 살면 되는 것 아닌가"라며 안도의 한숨을 내쉰 박 씨는 "딸의 이야기를 하면서


### 데이터셋 준비
심리상담을 위한 챗봇 데이터로 만들어보자

In [7]:
!git clone https://github.com/songys/Chatbot_data.git

Cloning into 'Chatbot_data'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 69 (delta 18), reused 4 (delta 4), pack-reused 48
Receiving objects: 100% (69/69), 398.65 KiB | 6.64 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [9]:
import pandas as pd
data = pd.read_csv('/content/Chatbot_data/ChatbotData.csv')

In [10]:
data.head(10)
# Qeustion 과 Answer 가 singleton 으로 연결되어 있는 데이터를 확인할 수 있음

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.,0
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.,0


In [11]:
added_special_token_num = tokenizer.add_special_tokens(['<s>', '</s>'])
print(added_special_token_num)
# tokenizer 에게 명시적으로 문장의 시작과 끝을 알려줌

2


그리고 fine-tuning 을 하는 과정을 거칠 예정이기 때문에 나중에 special token 이 추가가 된다고 하면 반드시 모델도 resize 하는 과정이 필요함

지금 task 에서는 특별한 다른 token 이 추가되지 않겠지만 만약 본인이 원하는 special token 이 추가가 된다면 반드시 추가된 token 이 몇개인지 기억해두고 나중에 모델을 resize 하는 과정을 필수로 필요로 함

In [12]:
pad_id = tokenizer.token_to_id("<pad>")
print(pad_id)
tokenizer.enable_padding(pad_id=pad_id, pad_token="<pad>")
tokenizer.enable_truncation(max_length=128)

# <pad> 토큰도 넣어주기

# BertTokenizer 에서는 padding 이라는 옵션으로 padding 을 설정해줬다면 이 tokenizer 는 enable_padding 으로 설정할 수 있음

# truncation 할 때도 enable_truncation 으로 max_length 가 몇개인지 명시적으로 알려줌으로써 truncation 과 padding 을 가능하게 만들어주게 됨

3


In [13]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, file_path):
        self.data = []
        self.file_path = file_path
        self.tokenizer = tokenizer

    def load_data(self):
        raw_data = pd.read_csv(self.file_path)
        train_data = '<s>'+raw_data['Q']+'</s>'+'<s>'+raw_data['A']+'</s>'
        #<s>안녕하세요</s><s> -> 네, 안녕하세요</s>
        tokenized_train_data = tokenizer.encode_batch(train_data)
        for single_data in tokenized_train_data:
            self.data.append(torch.tensor(single_data.ids).unsqueeze(0))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]
        return item


# 학습을 위한 데이터를 만들어줘야 하는데 이 과정은 GPT-2 에서 pre-training 을 위해 데이터를 만들어줬던거랑 똑같음
# load_data(), __get_item__() 함수를 구현함
# load_data() 는 실제로 dataset 에서 데이터를 load 하고 우리가 원하는대로 전처리가 이루어지도록 데이터를 수정하는 과정을 말함

In [15]:
train_dataset = ChatDataset(tokenizer=tokenizer, file_path='/content/Chatbot_data/ChatbotData.csv')
train_dataset.load_data()

# tokenizer 명시해주고 file_path 명시해 줌
# load_data() 하게 되면 전체 데이터들이 self.data 안에 저장됨

In [17]:
from torch.utils.data import DataLoader

data_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# 그 다음엔 떠먹여주는 data loader 를 만들어줌
# train dataset 을 batch_size 4만큼 input 으로 넣어라 라고 만들어주게 됨

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=1e-4, correct_bias=True)

epochs = 3

avg_loss = (0.0, 0.0)
for epoch in range(epochs):
    count=0
    for data in data_loader:
        optimizer.zero_grad()
        data = data.transpose(1,0)
        data = data.to('cpu')
        model = model.to('cpu')

        outputs = model(data, labels=data)
        loss, logits = outputs[:2]
        loss = loss.to('cpu')
        loss.backward()
        avg_loss = (avg_loss[0] * 0.99 + loss, avg_loss[1] * 0.99 + 1.0)
        optimizer.step()
        if count % 200 == 0:
            print('epoch no.{0}  train ({1}/{2})  loss = {3:.5f}  avg_loss = {4:.5f}' . format(epoch, count, len(data_loader), loss, avg_loss[0] / avg_loss[1]))
        count += 1

# 현재 예제는 torch 를 사용했음
# pre-training 에서 사용한 예제를 사용해도 똑같이 동작함
# 모델에 data 를 넣고 label 이 data 자체가 됨
# GPT-2 는 학습할 때 data input 을 한칸씩 넣어가면서 다음 토큰이 뭐가나오는지 그 확률이 최대가 되도록 학습이 됨

epoch no.0  train (0/2956)  loss = 2.68319  avg_loss = 2.68319


In [ ]:
torch.save(model.state_dict(), 'chitchat_model.bin')
# 모델을 저장하면 우리가 만든 chatbot 모델이 저장됨

In [ ]:
# 실제 inference 를 통해서 챗봇과 대화하는 task 를 해봐야 함
def encoding(text):
    text = '<s>'+text+'</s><s>'
    return torch.tensor(tokenizer.encode(text).ids).unsqueeze(0).to('cuda')

def decoding(ids):
    return tokenizer.decode_batch(ids)

tokenizer.no_padding()
tokenizer.no_truncation()

e_s = tokenizer.token_to_id('</s>')
unk = tokenizer.token_to_id('<unk>')

# encoding() 함수는 내가 입력한 text 에 대해 앞뒤로 bos, eos token 을 넣어주게 되고 그 다음에 생성을 해야하는 시점이 <s> 토큰으로 문장의 시작점을 알려준 것임
# 이 text 를 vocab_id 로 치환해주고 vocab_id 로 치환된 것이 모델의 input 으로 들어감
# 그럼 모델은 뒷부분에 생성된 단어들을 만들어낼거고 그걸 다시 decoding 함으로써 원래 정답을 알아낼 수 있음

In [ ]:
def get_answer(input_sent):
    input_ids = encoding(input_sent)

    sample_outputs = model.generate(
        input_ids,
        num_return_sequences=5,
        do_sample=True,
        max_length=128,
        top_k=50,
        top_p=0.95,
        eos_token_id=e_s,
        early_stopping=True,
        bad_words_ids=[[unk]]
    )

    decoded_result = decoding(sample_outputs.tolist())
    for result in decoded_result:
        print(result)

우리가 정의한 sentence 를 vocab_id 로 바꿔주고 그걸 모델의 입력으로 넣어주고 예제를 5개 보기 위해 설정하고 그리고 Sampling 도 넣고 top_k, top_p 두가지 옵션도 다 넣었고 max_length 까지 지정해줌
그 다음에 생성을 멈추도록 하는 eos_token_id 까지 입력해줌

여기에 추가로 bad_words_ids 라는 옵션이 추가로 나왔는데 가끔 <unk> 토큰을 생성할 수 있음
입력된 거에 대해서 적정한 Random Sampling 과 top_p, top_k 를 참고하다보니 <unk> 토큰이 등장할 수 있음

그 때 <unk> 토큰이 등장하게 되면 다른것을 선택해라라고 bad_words_ids 를 등록해줄 수 있음

여기에 배열형태로 넣게되면 그 token 들이 생성되지 않도록 피하는 과정이 generate 함수내에서 이뤄지게 됨

그래서 생성된 output(vocab_ids)을 이걸 decoding 을 하고 반환하게 되면 채팅결과가 나오는 것임

##### Test

In [ ]:
get_answer('안녕?')

In [ ]:
get_answer('만나서 반가워.')

In [ ]:
get_answer('인공지능의 미래에 대해 어떻게 생각하세요?')

In [ ]:
get_answer('여자친구 선물 추천해줘.')

In [ ]:
get_answer('앞으로 인공지능이 어떻게 발전하게 될까요?')

In [ ]:
get_answer('이제 그만 수업 끝내자.')